# Reproducing MIST for Short Text Clustering

## Introduction
The paper [MIST: Mutual Information Maximization for Short Text Clustering](https://aclanthology.org/2024.acl-long.610/) introduces a method to fine-tune a pre-trained BERT model by jointly optimizing a clustering loss and two representation learning losses. The authors have shared their source code on [GitHub](https://github.com/c4n/clustering_mist/).

The task is to visit the GitHub repository and compile a Colab notebook to replicate the experiments using the **SearchSnippets** dataset. If the dataset is too large for your Colab account, you may use the **AgNews** dataset or a subset of the original dataset.

**Note:** You are **not required to write your own code from scratch** for this assignment. Instead, focus on understanding and organizing the provided code. The main challenge is to make the provided code functional. This includes:
- Fixing any errors or dependencies required to run the code.
- Organizing the unstructured code into a clear and runnable sequence in the Colab notebook.
- Adapting the code to work with the chosen dataset (e.g., SearchSnippets, AgNews, or a subset).

Once you successfully run the code, present the results in the Colab notebook, including:
- ACC and NMI on the target dataset, as reported by the authors in Table 1.

In [2]:
import argparse

from sentence_transformers import SentenceTransformer
from sentence_transformers_local import models, SentenceTransformerSequential
from models.Transformers import SCCLBert
from learners.cluster import ClusterLearner

from dataloader.dataloader import augment_loader, augment_loader_split
from training import training

from utils.kmeans import get_kmeans_centers
from utils.randomness import set_global_random_seed
import torch

MODEL_CLASS = {
    "distil": 'distilbert-base-nli-stsb-mean-tokens',
    "robertabase": 'roberta-base-nli-stsb-mean-tokens',
    "robertalarge": 'roberta-large-nli-stsb-mean-tokens',
    "msmarco": 'distilroberta-base-msmarco-v2',
    "xlm": "xlm-r-distilroberta-base-paraphrase-v1",
    "bertlarge": 'bert-large-nli-stsb-mean-tokens',
    "bertbase": 'bert-base-nli-stsb-mean-tokens',
    "paraphrase": "paraphrase-mpnet-base-v2",
    "paraphrase-distil": "paraphrase-distilroberta-base-v2",
    "paraphrase-Tiny" : "paraphrase-TinyBERT-L6-v2"
}

parser = argparse.ArgumentParser()
# parser.add_argument('--gpuid', nargs="+", type=int, default=[0], help="The list of gpuid, ex:--gpuid 3 1. Negative value means cpu-only")
parser.add_argument('--seed', type=int, default=0, help="")
parser.add_argument('--print_freq', type=float, default=100, help="")  
parser.add_argument('--result_path', type=str, default='./results/')

parser.add_argument('--bert', type=str, default='paraphrase', help="")
#parser.add_argument('--bert', type=str, default='distil', help="")

parser.add_argument('--bert_model', type=str, default='bert-base-uncased', help="")
parser.add_argument('--note', type=str, default='_search_snippets_distil_lre-4_JSD', help="")

# Dataset
# stackoverflow/stackoverflow_true_text
parser.add_argument('--dataset', type=str, default='search_snippets', help="")
#parser.add_argument('--dataset', type=str, default='stackoverflow', help="")
# parser.add_argument('--data_path', type=str, default='./datasets/stackoverflow/')
parser.add_argument('--max_length', type=int, default=32)
parser.add_argument('--train_val_ratio', type=float, default= [0.9, 0.1])

# Data for train and test
# ###### AgNews
# parser.add_argument('--data_path', type=str, default='./datasets/')
# parser.add_argument('--dataname', type=str, default='agnewsdataraw-8000', help="")
# parser.add_argument('--dataname_val', type=str, default='agnewsdataraw-8000', help="")
# parser.add_argument('--num_classes', type=int, default=4, help="")
# ####### SearchSnippets
parser.add_argument('--data_path', type=str, default='./datasets/augmented/contextual_20_2col_bert/')
## parser.add_argument('--dataname', type=str, default='train_search_snippets.csv', help="")
## parser.add_argument('--dataname_val', type=str, default='test_search_snippets.csv', help="")
parser.add_argument('--dataname', type=str, default='search_snippets', help="")
parser.add_argument('--dataname_val', type=str, default='search_snippets', help="")
parser.add_argument('--num_classes', type=int, default=8, help="")
# # ###### StackOverFlow
# parser.add_argument('--data_path', type=str, default='./datasets/stackoverflow/')
# parser.add_argument('--dataname', type=str, default='stackoverflow', help="")
# parser.add_argument('--dataname_val', type=str, default='stackoverflow_', help="")
# parser.add_argument('--num_classes', type=int, default=20, help="")
# ###### Biomedical
# # parser.add_argument('--data_path', type=str, default='./datasets/biomedical/')
# parser.add_argument('--dataname', type=str, default='biomedical', help="")
# parser.add_argument('--dataname_val', type=str, default='biomedical', help="")
# parser.add_argument('--num_classes', type=int, default=20, help="")
# ######## Tweet
# parser.add_argument('--data_path', type=str, default='./datasets/')
# parser.add_argument('--dataname', type=str, default='tweet_remap_label', help="")
# parser.add_argument('--dataname_val', type=str, default='tweet_remap_label', help="")
# parser.add_argument('--num_classes', type=int, default=89, help="")
# ######## GoogleNewsTS
# parser.add_argument('--data_path', type=str, default='./datasets/')
# parser.add_argument('--dataname', type=str, default='TS', help="")
# parser.add_argument('--dataname_val', type=str, default='TS', help="")
# parser.add_argument('--num_classes', type=int, default=152, help="")
# ######## GoogleNewsT
# parser.add_argument('--data_path', type=str, default='./datasets/')
# parser.add_argument('--dataname', type=str, default='T', help="")
# parser.add_argument('--dataname_val', type=str, default='T', help="")
# parser.add_argument('--num_classes', type=int, default=152, help="")
# ######## GoogleNewsS
# parser.add_argument('--data_path', type=str, default='./datasets/')
# parser.add_argument('--dataname', type=str, default='S', help="")
# parser.add_argument('--dataname_val', type=str, default='S', help="")
# parser.add_argument('--num_classes', type=int, default=152, help="")

# Learning parameters
parser.add_argument('--lr', type=float, default=1e-6, help="") #learning rate
parser.add_argument('--lr_scale', type=int, default=100, help="")
parser.add_argument('--max_iter', type=int, default=3000)
parser.add_argument('--batch_size', type=int, default=256) #batch size

# CNN Setting
#parser.add_argument('--out_channels', type=int, default=768)
#parser.add_argument('--use_cnn', type=str, default='cnn_1')
#parser.add_argument('--use_cnn', type=str, default='cnn_3')
#parser.add_argument('--use_cnn', type=str, default='cnn_5')
#parser.add_argument('--use_cnn', type=str, default='cnn_7')
#parser.add_argument('--use_cnn', type=str, default='cnn_cat')
#parser.add_argument('--use_cnn', type=str, default='cnn_avg')

# Contrastive learning
parser.add_argument('--use_head', type=bool, default=False)
parser.add_argument('--use_normalize', type=bool, default=False)

parser.add_argument('--weighted_local', type=bool, default=False, help="")
#parser.add_argument('--normalize_method', type=str, default='inverse_prob', help="")
parser.add_argument('--normalize_method', type=str, default='none', help="")

parser.add_argument('--contrastive_local_scale', type=float, default=0.002)  #  unused!!!
parser.add_argument('--contrastive_global_scale', type=float, default=0.008) #  unused!!!
parser.add_argument('--temperature', type=float, default=0.5, help="temperature required by contrastive loss")
parser.add_argument('--base_temperature', type=float, default=0.1, help="temperature required by contrastive loss")

# Clustering
parser.add_argument('--clustering_scale', type=float, default=0.02) #scale of clustering loss
parser.add_argument('--use_perturbation', action='store_true', help="")
parser.add_argument('--alpha', type=float, default=1)

args = parser.parse_args(args=[])
# args.use_gpu = args.gpuid[0] >= 0
args.resPath = None
args.tensorboard = None

C:\Users\user\.conda\envs\NatureLanguageAnalyze\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]='4'

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print(torch.cuda.device_count())

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
1
NVIDIA GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
import timeit
from datetime import datetime,timezone
start = timeit.default_timer()
now_utc = datetime.now(timezone.utc)
print('Time UTC:', now_utc)

# resPath, tensorboard = setup_path(args)
# args.resPath, args.tensorboard = resPath, tensorboard
set_global_random_seed(args.seed)

# Dataset loader
train_loader = augment_loader(args)

# torch.cuda.set_device(args.gpuid[0])
# torch.cuda.set_device(device)

# Initialize cluster centers
# by performing k-means after getting embeddings from Sentence-BERT with mean-pooling(defualt)
sbert = SentenceTransformer(MODEL_CLASS[args.bert])
cluster_centers = get_kmeans_centers(sbert, train_loader, args.num_classes) 



# Model
# 1. Transformer model 
# use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
# word_embedding_model = models.Transformer(MODEL_CLASS[args.bert])

word_embedding_model = models.Transformer('sentence-transformers/paraphrase-mpnet-base-v2')
# word_embedding_model = models.Transformer('sentence-transformers/stanford-sentiment-treebank-roberta.2021-03-11')

# model = SentenceTransformer('distilbert-base-nli-mean-tokens')
dimension = word_embedding_model.get_word_embedding_dimension()
# word_embedding_model = torch.nn.DataParallel(word_embedding_model)


# 2. CNN model
# cnn = models.CNN(in_word_embedding_dimension = word_embedding_model.get_word_embedding_dimension(), 
#                  use_cnn = args.use_cnn, out_channels = word_embedding_model.get_word_embedding_dimension())

# 3. Pooling 
# pooling_model = models.Pooling(cnn.get_word_embedding_dimension(),
#                                pooling_mode_mean_tokens=True,
#                                pooling_mode_cls_token=False,
#                                pooling_mode_max_tokens=False)
pooling_model = models.Pooling(dimension,
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False, 
                               pooling_mode_weighted_tokens=False)

# 4. Feature extractor 
#feature_extractor = SentenceTransformerSequential(modules=[word_embedding_model, cnn, pooling_model])
feature_extractor = SentenceTransformerSequential(modules=[word_embedding_model, pooling_model], device = 'cuda')

# 5. main model
model = SCCLBert(feature_extractor, cluster_centers=cluster_centers, alpha = args.alpha, use_head = args.use_head)  


# Optimizer 
optimizer = torch.optim.Adam([
    {'params':word_embedding_model.parameters(), 'lr': args.lr*6},
#    {'params':cnn.parameters(), 'lr': args.lr*50},
    {'params':pooling_model.parameters()},
#    {'params':model.head.parameters(), 'lr': args.lr*args.lr_scale},
    {'params':model.cluster_centers, 'lr': args.lr*60}], lr=args.lr)
# # optimizer = torch.optim.Adam(lr=1e-4,params=model.parameters())
# optimizer = torch.optim.AdamW([
#     {'params':word_embedding_model.parameters(), 'lr': args.lr},
# #    {'params':cnn.parameters(), 'lr': args.lr*50},
#     {'params':pooling_model.parameters()},
# #    {'params':model.head.parameters(), 'lr': args.lr*args.lr_scale},
#     {'params':model.cluster_centers, 'lr': args.lr*20}], lr=args.lr)
# # optimizer = torch.optim.Adam(lr=1e-4,params=model.parameters())
print(optimizer)


# Set up the trainer    
learner = ClusterLearner(model, feature_extractor, optimizer, args.temperature, args.base_temperature,
                         args.contrastive_local_scale, args.contrastive_global_scale, args.clustering_scale, use_head = args.use_head, use_normalize = args.use_normalize)
# learner = torch.nn.DataParallel(learner)
learner = learner.cuda()

# split train - validation
if args.train_val_ratio != -1:
    train_loader, val_loader = augment_loader_split(args)
    training(train_loader, learner, args, val_loader = val_loader)
# normal
else:
    training(train_loader, learner, args) 

Time UTC: 2024-12-15 10:19:49.285107+00:00


C:\Users\user\.conda\envs\NatureLanguageAnalyze\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


all_embeddings:(12340, 768), true_labels:12340, pred_labels:12340
true_labels tensor([2, 0, 0,  ..., 3, 3, 1], dtype=torch.int32)
pred_labels tensor([7, 5, 5,  ..., 4, 3, 7], dtype=torch.int32)
Iterations:28, Clustering ACC:0.735, centers:(8, 768)


C:\Users\user\.conda\envs\NatureLanguageAnalyze\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


initial_cluster_centers =  torch.Size([8, 768])
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 6e-06
    maximize: False
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-06
    maximize: False
    weight_decay: 0

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5.9999999999999995e-05
    maximize: False
    weight_decay: 0
)
train_sample 0.9 11106
val_sample 0.1 1234

=3000/44=Iterations/Batches


...

C:\Users\user\.conda\envs\NatureLanguageAnalyze\lib\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


------------- Evaluate Training Set -------------
------------- 44 batches -------------
all_pred 8
step: 2900
[Representation] Clustering scores: {'NMI': 0.6850231383167724, 'ARI': 0.6376923759092498, 'AMI': 0.6846788311813125}
[Representation] ACC: 0.7672
[Representation] ACC sklearn: 0.1155
[Model] Clustering scores: {'NMI': 0.6848202552892154, 'ARI': 0.637528958999353, 'AMI': 0.6844757267764942}
[Model] ACC: 0.7672
[Model] ACC sklearn: 0.1015
------------- Evaluate Validation Set -------------
------------- 5 batches -------------
all_pred 8
step: 2900
[Representation] Clustering scores: {'NMI': 0.662678001070684, 'ARI': 0.6071011102450165, 'AMI': 0.6592784627408529}
[Representation] ACC: 0.7520
[Representation] ACC sklearn: 0.0122
[Model] Clustering scores: {'NMI': 0.662678001070684, 'ARI': 0.6071011102450165, 'AMI': 0.6592784627408529}
[Model] ACC: 0.7520
[Model] ACC sklearn: 0.0997
Time UTC: 2024-12-15 12:34:48.742482+00:00
Current running time 8102.97 seconds


C:\Users\user\.conda\envs\NatureLanguageAnalyze\lib\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


------------- Evaluate Training Set -------------
------------- 44 batches -------------
all_pred 8
step: 3000
[Representation] Clustering scores: {'NMI': 0.6850827872396028, 'ARI': 0.6379499905825005, 'AMI': 0.6847385464325233}
[Representation] ACC: 0.7659
[Representation] ACC sklearn: 0.4520
[Model] Clustering scores: {'NMI': 0.6850827872396028, 'ARI': 0.6379499905825005, 'AMI': 0.6847385464325233}
[Model] ACC: 0.7659
[Model] ACC sklearn: 0.0996
------------- Evaluate Validation Set -------------
------------- 5 batches -------------
all_pred 8
step: 3000
[Representation] Clustering scores: {'NMI': 0.618653925655043, 'ARI': 0.5612183679113015, 'AMI': 0.6148103675654887}
[Representation] ACC: 0.7318
[Representation] ACC sklearn: 0.3428
[Model] Clustering scores: {'NMI': 0.6658783871488453, 'ARI': 0.6111781831840079, 'AMI': 0.6625087207392094}
[Model] ACC: 0.7593
[Model] ACC sklearn: 0.0948
Time UTC: 2024-12-15 12:38:20.047799+00:00
Current running time 8314.27 seconds


C:\Users\user\.conda\envs\NatureLanguageAnalyze\lib\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


### The results:

```
------------- Evaluate Training Set -------------
------------- 44 batches -------------
all_pred 8
step: 3000
[Representation] Clustering scores: {'NMI': 0.6850827872396028, 'ARI': 0.6379499905825005, 'AMI': 0.6847385464325233}
[Representation] ACC: 0.7659
[Representation] ACC sklearn: 0.4520
[Model] Clustering scores: {'NMI': 0.6850827872396028, 'ARI': 0.6379499905825005, 'AMI': 0.6847385464325233}
[Model] ACC: 0.7659
[Model] ACC sklearn: 0.0996
------------- Evaluate Validation Set -------------
------------- 5 batches -------------
all_pred 8
step: 3000
[Representation] Clustering scores: {'NMI': 0.618653925655043, 'ARI': 0.5612183679113015, 'AMI': 0.6148103675654887}
[Representation] ACC: 0.7318
[Representation] ACC sklearn: 0.3428
[Model] Clustering scores: {'NMI': 0.6658783871488453, 'ARI': 0.6111781831840079, 'AMI': 0.6625087207392094}
[Model] ACC: 0.7593
[Model] ACC sklearn: 0.0948
Time UTC: 2024-12-15 12:38:20.047799+00:00
Current running time 8314.27 seconds
```

From the paper report: The performace at SearchSnippets dataset is 76.72 ACC and 67.69 NMI. The number we reproduce is 75.93 ACC and 66.5p NMI. The difference is not significant.